# Ganによる異常検知

In [4]:
!gpustat

/bin/sh: gpustat: command not found


In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [7]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms
%config IPCompleter.greedy=True

import matplotlib.pyplot as plt
%matplotlib inline

## 6-1 GANによる異常画像検知のメカニズム

### Ganを用いた異常画像検知の必要性
異常画像検知は医療の現場や，製造業の生産ラインで利用される．従来では熟練者の経験で行われていたものをマシンに行わせる．
従来のルールベースの手法ではうまくいかなったものがディープラーニングの導入によって補助・代替できる可能性がある．

**しかしディープラーニングを用いた異常画像検知の問題点としては正常画像の数に対して異常画像の数が圧倒的に少ないことである．** そこで本節では正常画像のみでディープラーニングを行って異常画像を検出できるアルゴリズムの構築することに取り組む．

### AnoGANの概要
最も自然な考えとしては正常画像のみを生成するGANを構築して，そのDiscriminatorに対してテストしたい画像を入力することで教師画像か偽画像かを判断させる方法で，教師画像と判断されればこれは正常画像であり，偽画像と判断されればこれは異常画像として検知される．しかしこのような方法では実際にはうまくいかないらしい．

実際にはGeneratorも用いて異常検知させる．大まかには以下の手順で行う．
1. **まず通常のGANと同様に正常画像のみでGANを構築する．**
2. **Generatorに入力する生成ノイズzの中で最もテストしたい画像に近い画像を生成できるzを決定する．**
3. **その決定したzを用いて生成した画像とテストしたい画像がどれくらい似ているかで異常検知を行う．**

次節ではこれらの手順に加えて以下についても理解していく．
- Discriminatorはどこで用いるのか
- 生成ノイズzの最適値はどのように決定するのか
- 生成画像とテスト画像の似ている度合いはどのように判断するのか

## 6-2 AnoGANの実装と異常検知の実施

### DCGANの学習
AnoGANのGANには前節で実装したDCGANを用いる．

DCGANに対してAnoGAN用にDiscriminatorの最後の出力の１つ手前の層の特徴量も出力するように変更する．
この特徴量を用いて生成ノイズの最適値を求める．

In [6]:
class Generator(nn.Module):
    
    def __init__(self, z_dim = 20, image_size=64):
        super(Generator, self).__init__()
        
        # ここで out_channels = image_size*8に特別な意味は？
        self.layer1 = nn.Sequential(
            nn.ConvTranspose2d(z_dim, image_size*8, kernel_size=4, stride=1),
            nn.BatchNorm2d(image_size*8),
            nn.ReLU(inplace=True))
        
        self.layer2 = nn.Sequential(
            nn.ConvTranspose2d(image_size*8, image_size*4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(image_size*4),
            nn.ReLU(inplace=True))
            
        self.layer3 = nn.Sequential(
            nn.ConvTranspose2d(image_size*4, image_size*2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(image_size*2),
            nn.ReLU(inplace=True))
              
        self.layer4 = nn.Sequential(
            nn.ConvTranspose2d(image_size*2, image_size, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(image_size),
            nn.ReLU(inplace=True))
        
        self.last = nn.Sequential(
            nn.ConvTranspose2d(image_size, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh())
        # 白黒なので出力は1チャネル
        
        
    def forward(self, z):
        out = self.layer1(z)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.last(out)
        
        return out

In [8]:
class Discriminator(nn.Module):
    
    def __init__(self, z_dim=20, image_size=64):
        super(Discriminator, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, image_size, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))

        self.layer2 = nn.Sequential(
            nn.Conv2d(image_size, image_size*2, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))

        self.layer3 = nn.Sequential(
            nn.Conv2d(image_size*2, image_size*4, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))

        self.layer4 = nn.Sequential(
            nn.Conv2d(image_size*4, image_size*8, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))
        
        self.last = nn.Conv2d(image_size*8, 1, kernel_size=4, stride=1)
        
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        """
        生成ノイズzを決定するための特徴量を出力する
        """
        feature = out
        feature = feature.view(f)
        
        out = self.last(out)
        
        return out